<a href="https://colab.research.google.com/github/Reynxzz/capstone-ipb/blob/main/Capstone_IPB_Making_Dataset_%26_Recommender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Making Dataset

STEP bikin dataset:
1. Bikin random point di google earth sekitaran IPB
2. Export latitude dan longitude dari point2 itu
3. Ambil data elevation per point tersebut pake Open Topodata API
4. Generate random route (cyclic) dengan start Agrimart dan Rektorat
5. Sesuaikan distancenya (max 3 km dan max 5 km)
6. Sesuaikan kesulitannya pake data elevasi (Easy, Medium, Hard) --> lagi disini
7. Sesuaikan dengan pemandangan

Expected dataset:

| Route | (lat,lon) | Distance | Duration | Elevation Max | Elevation Min | Elevation Difference | Difficulty | Scenery | Preview(image) |

###Import CSV datasets from Google Earth

In [1]:
import csv

# Open the CSV file for reading
with open('/content/drive/My Drive/Capstone IPB/Tempat_di_IPB.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)

    # Create a list of tuples containing latitude and longitude
    coordinates = [(float(row['Y']), float(row['X'])) for row in reader]

Create dataframe

In [40]:
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/Capstone IPB/Tempat_di_IPB.csv')

In [41]:
# remove uneccessary column
df = df.drop(df.columns[-1], axis=1)

# change column name
df = df[['Name', 'Y', 'X']]
df = df.rename(columns={'Name': 'name', 'X': 'longitude', 'Y': 'latitude'})
df

,name,latitude,longitude
0,Starting Point 1 - Agrimart,-6.559847,106.731220
1,Depan Koin IPB,-6.560860,106.727949
2,Depan Rektorat,-6.560678,106.725458
3,Pintu Masuk ke Kiri,-6.557232,106.719038
4,Warteg Sebelah Alhur,-6.556168,106.724086
5,Depan Alhur,-6.556064,106.725050
6,Perumdos 1,-6.552980,106.721197
7,Belakang 1,-6.552237,106.717024
8,Sawah 1,-6.551354,106.714950
9,Belakang Banget,-6.547448,106.715541


Visualize Dataframe

In [42]:
# create a map centered on the first location
m = folium.Map(location=[df['latitude'][0], df['longitude'][0]], zoom_start=10)

# add markers for each location in the DataFrame
for index, row in df.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']], popup=row['name']).add_to(m)

# display the map
m

###Generate Routes

Bikin rute random

In [32]:
import random
import json
import folium
import requests

def routes_maker(total_point, max_distance):
    subset = []
    # define start and end coordinates
    start = coordinates[0]
    end = coordinates[0]

    # randomly select a subset of coordinates
    subset = random.sample(coordinates, k=total_point)  # k is the number of coordinates in the subset

    # add the start and end coordinates to the beginning and end of the subset
    subset.insert(0, start)
    subset.append(end)

    api_url = 'https://api.openrouteservice.org/v2/directions/foot-walking'
    api_key = '5b3ce3597851110001cf6248d4725ac662f8461eb589679758fb20ea'

    locations = [list(r) for r in subset]
    total_distance = 0
    total_duration = 0

    for i in range(len(locations)-1):
        # Define the start and end coordinates for the API call
        start_coords = f"{locations[i][1]},{locations[i][0]}"
        end_coords = f"{locations[i+1][1]},{locations[i+1][0]}"
        
        # Define the API call parameters, including the start and end coordinates, the API key, and the travel mode (walking)
        params = {
            'api_key': api_key,
            'start': start_coords,
            'end': end_coords,
            'profile': 'foot-walking',
        }
        
        # Make the API call and parse the response as JSON
        response = requests.get(api_url, params=params)
        data = json.loads(response.text)
        # print(data)
        # Extract the walking distance and duration from the response
        distance = data['features'][0]['properties']['segments'][0]['distance']
        duration = data['features'][0]['properties']['segments'][0]['duration']
        # elevation = data['features'][0]['properties']['segments'][0]['steps'][0]['elevation']

        print(f"Distance between locations {i+1} and {i+2}: {distance:.2f} meters")
        # elevation_profile = [(step['distance'], step['elevation']) for step in elevation]
        # print(elevation_profile)
        # total distance and duration
        total_distance += distance
        total_duration += duration
        
        # visualize route
        m = folium.Map(location=start, zoom_start=14)
        folium.PolyLine(locations=[list(reversed(coord)) 
                              for coord in 
                              data['features'][0]['geometry']['coordinates']]).add_to(m)

    print(f"Total distance: {total_distance:.2f} meters")
    print(f"Total duration: {total_duration:.2f} seconds")
    
    folium.Marker(location=start, popup='Start here', icon=folium.Icon(color='green', prefix='fa', icon='running')).add_to(m)
    folium.Marker(location=locations[len(locations) // 2], popup='Finish here', icon=folium.Icon(color='blue',prefix='fa', icon='undo')).add_to(m)

    if total_distance < max_distance + 300:
      return total_distance, m, subset 
    else:      
      return routes_maker(total_point, max_distance)

In [33]:
total_distances, m, subset = routes_maker(2, 5000)

Distance between locations 1 and 2: 765.40 meters
Distance between locations 2 and 3: 1942.00 meters
Distance between locations 3 and 4: 2041.40 meters
Total distance: 4748.80 meters
Total duration: 3419.20 seconds


In [12]:
total_distances, subset

(3783.7,
 [(-6.55984667674852, 106.731220037341),
  (-6.55723238992818, 106.719037577124),
  (-6.55535606112686, 106.717589757062),
  (-6.55984667674852, 106.731220037341)])

Elevation Max, Min, Difference

In [13]:
def get_elevations(coordinates):
    """Lihat dokumentasi disini bro https://www.opentopodata.org/"""

    # Build the API URL for the OpenTopo API
    url = "https://api.opentopodata.org/v1/aster30m"

    # Make the API call for each coordinate and extract the elevation data
    elevations = []
    for coord in coordinates:
        lat, lon = coord
        query = f"{url}?locations={lat},{lon}"
        response = requests.get(query)
        if response.status_code == 200:
            data = response.json()
            elevation = data["results"][0]["elevation"]
            elevations.append(elevation)
        else:
            elevations.append(None)

    # Return the list of elevations
    return elevations

In [ ]:
# elevations = get_elevations(coordinates)

# # Print the list of elevations
# print(elevations)

[184.0, 180.0, 175.0, 176.0, 182.0, 184.0, 184.0, 167.0, 152.0, 135.0, 124.0, 138.0, 165.0, 166.0]


In [14]:
subset_elevations = get_elevations(subset)
subset_elevations

[184.0, 176.0, 166.0, 184.0]

In [15]:
max_elevation = max(subset_elevations)
min_elevation = min(subset_elevations)
elevation_difficulty = max_elevation - min_elevation

max_elevation, min_elevation, elevation_difficulty

(184.0, 166.0, 18.0)

Visualisasi rute

In [34]:
m